## General Preparation
import libs

load training data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from dl_utils.data_utils import load_CIFAR10

%matplotlib inline
plt.rcParams["figure.figsize"] = (10.0, 8.0)  # set default size of plots
plt.rcParams["image.interpolation"] = "nearest"
plt.rcParams["image.cmap"] = "gray"


def get_CIFAR10_data(
    num_training=49000, num_validation=1000, num_test=1000, num_dev=500
):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the linear classifier. These are the same steps as we used for the
    SVM, but condensed to a single function.
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = "/home/slipstream/Projects/dl/dl_utils/datasets/cifar-10-batches-py"

    # Cleaning up variables to prevent loading data multiple times (which may cause memory issue)
    try:
        del X_train, y_train
        del X_test, y_test
        print("Clear previously loaded data.")
    except:
        pass

    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

    # subsample the data
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]
    mask = np.random.choice(num_training, num_dev, replace=False)
    X_dev = X_train[mask]
    y_dev = y_train[mask]

    # Preprocessing: reshape the image data into rows
    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    X_val = np.reshape(X_val, (X_val.shape[0], -1))
    X_test = np.reshape(X_test, (X_test.shape[0], -1))
    X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))

    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis=0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    X_dev -= mean_image

    # add bias dimension and transform into columns
    X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
    X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
    X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
    X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])

    return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev


X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print("Train data shape: ", X_train.shape)
print("Train labels shape: ", y_train.shape)
print("Validation data shape: ", X_val.shape)
print("Validation labels shape: ", y_val.shape)
print("Test data shape: ", X_test.shape)
print("Test labels shape: ", y_test.shape)
print("dev data shape: ", X_dev.shape)
print("dev labels shape: ", y_dev.shape)

Train data shape:  (49000, 3073)
Train labels shape:  (49000,)
Validation data shape:  (1000, 3073)
Validation labels shape:  (1000,)
Test data shape:  (1000, 3073)
Test labels shape:  (1000,)
dev data shape:  (500, 3073)
dev labels shape:  (500,)


## Softmax Loss Implementation


In [2]:
# Complete the implementation of softmax_loss_naive and implement a (naive)
# version of the gradient that uses nested loops.
from builtins import range
import time


def softmax_loss_naive(W, X, y, reg):
    """
    Softmax loss function, naive implementation (with loops)

    Inputs have dimension D, there are C classes, and we operate on minibatches
    of N examples.

    Inputs:
    - W: A numpy array of shape (D, C) containing weights.
    - X: A numpy array of shape (N, D) containing a minibatch of data.
    - y: A numpy array of shape (N,) containing training labels; y[i] = c means
      that X[i] has label c, where 0 <= c < C.
    - reg: (float) regularization strength, which contributes to the loss by a
      regularization term: 0.5 * reg * np.sum(W * W)

    Returns a tuple of:
    - loss as single float
    - gradient with respect to weights W; an array of same shape as W
    """
    # Initialize the loss and gradient to zero.
    loss = 0.0
    dW = np.zeros_like(W)

    ##########################################################################
    # TODO: Compute the softmax loss and its gradient using explicit loops.  #
    # Store the loss in loss and the gradient in dW.                         #
    ##########################################################################
    # *****START OF YOUR CODE *****

    C = W.shape[1]
    N, D = X.shape
    for i in range(N):
        scores = X[i].dot(W)
        scores -= np.max(scores)
        probs = np.exp(scores) / np.sum(np.exp(scores))
        loss += -np.log(probs[y[i]])

        for j in range(C):
            dW[:, j] += (probs[j] - (j == y[i])) * X[i]

    loss /= N
    dW /= N
    loss += 0.5 * reg * np.sum(W * W)
    dW += reg * W

    # *****END OF YOUR CODE *****

    return loss, dW


for _ in range(10):
    # Generate a random softmax weight matrix and use it to compute the loss.
    W = np.random.randn(3073, 10) * 0.0001
    loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)
    print("loss: %f" % loss)

print("the correct loss should be close to: %f" % (-np.log(0.1)))  # (why?)

loss: 2.344710
loss: 2.325581
loss: 2.411054
loss: 2.313342
loss: 2.352015
loss: 2.372320
loss: 2.380199
loss: 2.328782
loss: 2.357075
loss: 2.385624
the correct loss should be close to: 2.302585


In [6]:
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# Use numeric gradient checking as a debugging tool.
# The numeric gradient should be close to the analytic gradient.
from dl_utils.gradient_check import grad_check_sparse

f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

# similar to SVM case, do another gradient check with regularization
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 5e1)
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 5e1)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

numerical: 4.938181 analytic: 4.938181, relative error: 2.126411e-08
numerical: -0.953480 analytic: -0.953480, relative error: 3.017346e-08
numerical: -1.029442 analytic: -1.029442, relative error: 2.733967e-08
numerical: 3.085417 analytic: 3.085417, relative error: 1.567481e-08
numerical: 0.118929 analytic: 0.118929, relative error: 3.830553e-07
numerical: -2.967244 analytic: -2.967244, relative error: 1.455346e-08
numerical: 0.573974 analytic: 0.573974, relative error: 5.591943e-08
numerical: 0.973372 analytic: 0.973372, relative error: 5.293806e-09
numerical: 2.394666 analytic: 2.394666, relative error: 4.340752e-08
numerical: 0.227554 analytic: 0.227554, relative error: 3.323408e-07
numerical: 2.748838 analytic: 2.748838, relative error: 3.483465e-08
numerical: 0.118313 analytic: 0.118313, relative error: 4.194923e-07
numerical: -3.410575 analytic: -3.410575, relative error: 9.752059e-09
numerical: 1.049818 analytic: 1.049817, relative error: 6.199807e-08
numerical: -2.576451 analy

### Optional

In [7]:
# Now implement a vectorized version in softmax_loss_vectorized.
def softmax_loss_vectorized(W, X, y, reg):
    """
    Softmax loss function, vectorized version.
    Inputs and outputs are the same as softmax_loss_naive.
    """
    # Initialize the loss and gradient to zero.
    loss = 0.0
    dW = np.zeros_like(W)

    #############################################################################
    # TODO: Compute the softmax loss and its gradient using no explicit loops.  #
    # Store the loss in loss and the gradient in dW.                            #
    #############################################################################
    # *****START OF YOUR CODE *****

    C = W.shape[1]
    N, D = X.shape

    scores = X @ W
    scores -= np.max(scores, axis=1, keepdims=True)
    probs = np.exp(scores) / np.sum(np.exp(scores), axis=1, keepdims=True)
    loss = np.sum(-np.log(probs[np.arange(N), y]))

    dW = X.T @ (probs - np.eye(C)[y])

    loss /= N
    dW /= N
    loss += 0.5 * reg * np.sum(W * W)
    dW += reg * W

    # *****END OF YOUR CODE *****

    return loss, dW


# The two versions (naive and vectorized) should compute the same results, but the vectorized version should be
# much faster.
tic = time.time()
loss_naive, grad_naive = softmax_loss_naive(W, X_dev, y_dev, 0.000005)
toc = time.time()
print("naive loss: %e computed in %fs" % (loss_naive, toc - tic))

tic = time.time()
loss_vectorized, grad_vectorized = softmax_loss_vectorized(W, X_dev, y_dev, 0.000005)
toc = time.time()
print("vectorized loss: %e computed in %fs" % (loss_vectorized, toc - tic))

# use the Frobenius norm to compare the two versions of the gradient.
grad_difference = np.linalg.norm(grad_naive - grad_vectorized, ord="fro")
print("Loss difference: %f" % np.abs(loss_naive - loss_vectorized))
print("Gradient difference: %f" % grad_difference)

naive loss: 2.385624e+00 computed in 0.026582s
vectorized loss: 2.385624e+00 computed in 0.004051s
Loss difference: 0.000000
Gradient difference: 0.000000


# Softmax Classifier

In [8]:
class Softmax:
    def __init__(self):
        self.W = None

    def train(
        self,
        X,
        y,
        learning_rate=1e-3,
        reg=1e-5,
        num_iters=100,
        batch_size=200,
        verbose=False,
    ):
        """
        Train this classifier using stochastic gradient descent.

        Inputs:
        - X: A numpy array of shape (N, D) containing training data; there are N
          training samples each of dimension D.
        - y: A numpy array of shape (N,) containing training labels; y[i] = c
          means that X[i] has label 0 <= c < C for C classes.
        - learning_rate: (float) learning rate for optimization.
        - reg: (float) regularization strength.
        - num_iters: (integer) max number of steps to take when optimizing
        - batch_size: (integer) number of training examples to use at each step.
        - verbose: (boolean) If true, print progress during optimization.

        Outputs:
        A list containing the value of the loss function at each training iteration.
        """
        num_train, dim = X.shape
        num_classes = (
            np.max(y) + 1
        )  # assume y takes values 0...K-1 where K is number of classes
        if self.W is None:
            # lazily initialize W
            self.W = 0.001 * np.random.randn(dim, num_classes)

        # Run stochastic gradient descent to optimize W
        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None

            #########################################################################
            # TODO:                                                                 #
            # Sample batch_size elements from the training data and their           #
            # corresponding labels to use in this round of gradient descent.        #
            # Store the data in X_batch and their corresponding labels in           #
            # y_batch; after sampling X_batch should have shape (batch_size, dim)   #
            # and y_batch should have shape (batch_size,)                           #
            #                                                                       #
            # Hint: Use np.random.choice to generate indices.                       #
            #########################################################################
            # *****START OF YOUR CODE *****

            idx = np.random.choice(num_train, batch_size)
            X_batch = X[idx]
            y_batch = y[idx]

            # perform parameter update
            #########################################################################
            # TODO:                                                                 #
            # Update the weights self.W using the gradient and the learning rate.   #
            #########################################################################
            # *****START OF YOUR CODE *****

            loss, grad = self.loss(X_batch, y_batch, reg)
            self.W -= learning_rate * grad

            # *****END OF YOUR CODE *****

            if verbose and it % 100 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))

        return loss_history

    def predict(self, X):
        """
        Use the trained weights of this linear classifier to predict labels for
        data points.

        Inputs:
        - X: A numpy array of shape (N, D) containing training data; there are N
          training samples each of dimension D.

        Returns:
        - y_pred: Predicted labels for the data in X. y_pred is a 1-dimensional
          array of length N, and each element is an integer giving the predicted
          class.
        """
        y_pred = np.zeros(X.shape[0])
        ###########################################################################
        # TODO:                                                                   #
        # Implement this method. Store the predicted labels in y_pred.            #
        ###########################################################################
        # *****START OF YOUR CODE *****

        scores = X @ self.W
        y_pred = np.argmax(scores, axis=1)

        # *****END OF YOUR CODE *****
        return y_pred

    def loss(self, X_batch, y_batch, reg):
        # return softmax_loss_vectorized(self.W, X_batch, y_batch, reg)
        return softmax_loss_naive(self.W, X_batch, y_batch, reg)

In [9]:
# Train the Softmax Classifer on the CIFAR10 dataset

# Provided as a reference. You may change these hyperparameters
learning_rates = 1e-6
regularization_strengths = 1e3

softmax = Softmax()
softmax.train(
    X_train,
    y_train,
    learning_rates,
    regularization_strengths,
    num_iters=1000,
    verbose=True,
)

# Evaluate on test set
y_test_pred = softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print("softmax on raw pixels final test set accuracy: %f" % (test_accuracy,))

iteration 0 / 1000: loss 20.882288
iteration 100 / 1000: loss 15.368358
iteration 200 / 1000: loss 12.335218
iteration 300 / 1000: loss 10.312426
iteration 400 / 1000: loss 8.599302
iteration 500 / 1000: loss 7.398833
iteration 600 / 1000: loss 6.241856
iteration 700 / 1000: loss 5.583079
iteration 800 / 1000: loss 4.703070
iteration 900 / 1000: loss 4.037365
softmax on raw pixels final test set accuracy: 0.370000
